# P13 

# Class Homeworks Analysis

Presentado por:

Diego Alejandro Diaz - 202027745 Andres Felipe Botero - 201421261 Sindy Carolina Niño - 202020034 Juliana López Botero - 202027785

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

In [2]:
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
# from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [3]:
df = pd.read_excel("C:/Users/ddiaz/OneDrive - Universidad de los Andes/Intersemestral/Modelos analíticos avanzados/E13/Consolidado_respuestas_escribir_v2.xlsx")

In [4]:
df.columns=["ID","Genero","Pregrado","Sector","Edad","E1","E4","E6","E8","E10","E12"]

In [5]:
df['E1'] = df['E1'].astype(str)
df['E4'] = df['E4'].astype(str)
df['E6'] = df['E6'].astype(str)
df['E8'] = df['E8'].astype(str)
df['E10'] = df['E10'].astype(str)
df['E12'] = df['E12'].astype(str)

In [45]:
df

,ID,Genero,Pregrado,Sector,Edad,E1,E4,E6,E8,E10,E12,y1,x1,y2
0,1,M,Matemáticas,Estudio de mercados,30.0,Dentro de las diferentes aplicaciones que tien...,La principal diferencia entre las series de ti...,Los arboles de decisión son herramienta de Mac...,Al proponer un método para realizar una predic...,Durante el desarrollo de soluciones dentro de ...,La empresa kore.ai se especializa en la implem...,0,Dentro de las diferentes aplicaciones que tien...,4
1,2,M,NaN,nan,NaN,nan,una serie de tiempo se enfoca en un individio ...,nan,nan,Un micro servicio es una parte de la arquitect...,nan,0,nan una serie de tiempo se enfoca en un indivi...,0
2,3,M,NaN,nan,NaN,Mercados energeticos: Tras la liberación y des...,Los datos de series de tiempo son un conjunto ...,nan,Los metodos de ensamble nos permiten mejorar e...,Los microservicios defininen una nueva metodol...,nan,0,Mercados energeticos: Tras la liberación y des...,0
3,4,M,Estadistica,Financiero,28.0,Las series de tiempo son una colección de obse...,Los datos de series de tiempo son un conjunto ...,Un árbol de decisión es una especie de mapa en...,Los métodos de ensamble de modelos o métodos c...,Los microservicios son tanto un estilo de arqu...,La función de autocompletar las palabras en Wh...,0,Las series de tiempo son una colección de obse...,2
4,5,M,NaN,nan,NaN,"1. Actualmente, la empresa de mascotas Laika q...",Un conjunto de datos de una serie de tiempo co...,nan,Los modelos de ensamble consisten en mezclar l...,Los microservicios son tanto un estilo de arqu...,nan,0,"1. Actualmente, la empresa de mascotas Laika q...",0
5,6,M,Estadística,Público,30.0,Las series de tiempo desempeñan un papel funda...,Las series tiempo son una secuencia de datos o...,Los árboles de decisión son uno de los modelos...,nan,Los microservicios son un tipo de arquitectura...,\nGPT-2 - OpenAI\nGenerative Pre-trained Trans...,0,Las series de tiempo desempeñan un papel funda...,3
6,7,M,Ingeniería de Sistemas,Educación,33.0,1. Predicción de precios de acciones\n2. Análi...,La diferencia entre las series de tiempo y los...,Los árboles de decisión son modelos de predicc...,nan,Los microservicios es una metodología de desar...,https://www.grammarly.com/\nGrammarly\nEsta ap...,0,1. Predicción de precios de acciones\n2. Análi...,4
7,8,M,NaN,nan,NaN,"Precio de acciones en la bolsa, sujeto a las t...",Una serie de tiempo es un conjunto de datos or...,nan,nan,A nivel general los Microservicios son pequeño...,nan,0,"Precio de acciones en la bolsa, sujeto a las t...",0
8,9,M,Ingeniería Electrónica,Consultoría de negocios y Tecnología,44.0,Pronóstico del tiempo: Dado el comportamiento ...,La diferencia entre series de tiempo y datos p...,Los tipos de árboles de decisión se basan en e...,Hay tres razones principales por las que los m...,Los microservicios son servicios desplegados i...,La Fiscalía General de la Nación implementó en...,0,Pronóstico del tiempo: Dado el comportamiento ...,3
9,10,M,Finanzas,Fintech,25.0,1) Comportamiento del precio del dólar. Usando...,The main difference between panel data and tim...,nan,Los modelos de ensambles son una estrategia ex...,Un microservicio es una forma de desarrollo en...,Whale & Jaguar es una compañía que se especial...,0,1) Comportamiento del precio del dólar. Usando...,2


# Exercise 13.1

Analyze the writing patterns of each student

Para evaluar los patrones de escritura de cada estudiante, se consolidan todos los textos escritos en uno solo

In [9]:
df['x1'] = df['E1'] + ' ' + df['E4'] + ' ' + df['E6'] + ' ' + df['E8'] + ' ' + df['E10'] + ' ' + df['E12'] 

In [10]:
X = df.x1
X

0     Dentro de las diferentes aplicaciones que tien...
1     nan una serie de tiempo se enfoca en un indivi...
2     Mercados energeticos: Tras la liberación y des...
3     Las series de tiempo son una colección de obse...
4     1. Actualmente, la empresa de mascotas Laika q...
5     Las series de tiempo desempeñan un papel funda...
6     1. Predicción de precios de acciones\n2. Análi...
7     Precio de acciones en la bolsa, sujeto a las t...
8     Pronóstico del tiempo: Dado el comportamiento ...
9     1) Comportamiento del precio del dólar. Usando...
10    1.\tUna planta lleva el conteo de su producció...
11    El primer ejemplo corresponde al registro mens...
12    1. Contactos Call Center: El número de contact...
13    Ejemplo 1: Cantidad de usuarios nuevos que se ...
14    Los siguientes son tres ejemplos en los cuales...
15    1.\tPara anticipar el futuro de una empresa ma...
16    Uno de las principales aplicaciones del anális...
17    Los análisis de las series de tiempo son ú

Utilizando el método TF-IDF, se construye una matriz para cada estudiante

In [23]:
from nltk.corpus import stopwords
spanish_stopwords = stopwords.words('spanish')

In [60]:
# create a document-term matrix using TF-IDF
vect = TfidfVectorizer(ngram_range=(1, 4),lowercase=False,stop_words=spanish_stopwords)
dtm = vect.fit_transform(X)
features = vect.get_feature_names()
dtm.shape

(38, 77597)

Utilizando hasta 4 ngrams, quitando las mayúsculas y removiendo las stopwords, se obtuvieron 77597 tokens. A continuación se muestran las 10 palabras con mejor score TF-IDF para cada uno de los estudiantes.

In [61]:
for i in range(dtm.shape[0]):
    feature_array = np.array(features)
    tfidf_sorting = np.argsort(dtm[i].toarray()).flatten()[::-1]

    n = 10
    top_n = feature_array[tfidf_sorting][:n]
    
    print(f'ID {i}:', top_n, '\n')

ID 0: ['diferentes' 'micro' 'permite' 'medio' 'micro servicios' 'ai' 'viento'
 'tipos' 'diferentes tipos' 'soluciones'] 

ID 1: ['mas' 'través apis' 'apis' 'manera' 'micro' 'analíticos'
 'través apis cada' 'mas efectiva' 'apis cada' 'permite organización'] 

ID 2: ['datos' 'tiempo' 'datos series' 'energía' 'modelos' 'series' 'Los datos'
 'series tiempo' 'mercados' 'muestra'] 

ID 3: ['pronósticos' 'Los' 'conjunto' 'Los pronósticos' 'mitad' 'mitad votos'
 'decisión' 'datos' 'microservicios' 'votos'] 

ID 4: ['tiempo' 'contiene múltiples' 'items' 'varios modelos diferentes'
 'contiene múltiples observaciones' 'través tiempo' 'modelos'
 'múltiples observaciones' 'modelos diferentes' 'datos'] 

ID 5: ['GPT' 'tiempo' 'OpenAI' 'periodos tiempo' 'manteniendo' 'periodos'
 'Administrativo Nacional Estadística DANE'
 'Administrativo Nacional Estadística' 'Administrativo Nacional'
 'calculada Departamento Administrativo Nacional'] 

ID 6: ['aplicativo' 'Serie tiempo' 'Datos panel' 'Serie' 'datos'

# Exercise 13.2
Create a classifier to predict the gender of each student

Se construye la variable respuesta

In [7]:
y1 = df.Genero
y1.describe()

count     38
unique     2
top        M
freq      28
Name: Genero, dtype: object

In [8]:
df['y1'] = df['Genero'].replace(['F','M'], [1,0])

In [62]:
y1 = df.y1
y1.describe()

count    38.000000
mean      0.263158
std       0.446258
min       0.000000
25%       0.000000
50%       0.000000
75%       0.750000
max       1.000000
Name: y1, dtype: float64

Utilizando la tokenización anterior, se construye un Random Forest para predecir el género de los estudiantes basados en sus respuestas a los ejercicios de clase

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [63]:
# define a function that accepts a vectorizer and calculates the accuracy
def tokenize_test(vect):
    X_dtm = vect.fit_transform(X)
    print('Features: ', X_dtm.shape[1])
    rf = RandomForestClassifier()
    print(pd.Series(cross_val_score(rf, X_dtm, y, cv=10)).describe())

In [64]:
X_dtm = vect.fit_transform(X)
rf = RandomForestClassifier()
pd.Series(cross_val_score(rf, X_dtm, y1, cv=10)).mean()

0.7333333333333334

Aplicando el cross validation sobre 10 folds, se obtuvo un score de 0.733

# Exercise 13.3
Create a classifier to predict the industry in which each student works

In [40]:
df['Sector'] = df['Sector'].astype(str)

In [41]:
Industry = df.groupby(['Sector']).agg({'ID':'count'}).reset_index()

In [42]:
Industry

,Sector,ID
0,Consultoría de negocios y Tecnología,1
1,Consumo Masivo,1
2,Consumo masivo,1
3,Cultura,1
4,Data Scientist - Banca,1
5,Educación,1
6,Estudio de mercados,1
7,FINANCIERO,1
8,Financiero,9
9,Fintech,2


Hay 18 distintos sectores registrados en los estudiantes, varios de ellos con sólo un participante. Para poder realizar un modelo más ajustado al tamaño de la muestra, se agrupan los sectores en 5 categorías.

- nan = 0
- Consumo Masivo = 1
- Financiero = 2
- Otros = 3
- Servicios = 4

In [43]:
def f(row):
    if row['Sector'] == 'nan':
        val = 0
    elif row['Sector'] == 'Consumo masivo' or row['Sector'] == 'Consumo Masivo':
        val = 1
    elif row['Sector'] == 'FINANCIERO' or row['Sector'] == 'Financiero' or row['Sector'] == 'Fintech' or row['Sector'] == 'Data Scientist - Banca' or row['Sector'] == 'Pensiones':
        val = 2
    elif row['Sector'] == 'Operaciones' or row['Sector'] == 'Cultura' or row['Sector'] == 'Público' or row['Sector'] == 'Consultoría  de negocios y Tecnología':
        val = 3
    else:
        val = 4
    return val

In [44]:
df['y2'] = df.apply(f, axis=1)

In [70]:
y2=df.y2

In [48]:
IndustryY = df.groupby(['y2']).agg({'ID':'count'}).reset_index()

In [49]:
IndustryY

,y2,ID
0,0,11
1,1,2
2,2,14
3,3,4
4,4,7


Con las 5 categorías construidas se realizará un modelo multiclass

In [72]:
X_dtm = vect.fit_transform(X)
nb = MultinomialNB()
pd.Series(cross_val_score(nb, X_dtm, y2, cv=2)).mean()

0.3684210526315789

Se obtiene un score de 0.368, esto debido al pequeño tamaño de la muestra y la poca cantidad de estudiantes con Sector = 1 (Consumo Masivo)

Se categorizará el consumo masivo dentro del grupo 3 y se correrá nuevamente el modelo

In [76]:
def f2(row):
    if row['Sector'] == 'nan':
        val = 0
    elif row['Sector'] == 'FINANCIERO' or row['Sector'] == 'Financiero' or row['Sector'] == 'Fintech' or row['Sector'] == 'Data Scientist - Banca' or row['Sector'] == 'Pensiones':
        val = 1
    elif row['Sector'] == 'Consumo masivo' or row['Sector'] == 'Consumo Masivo' or row['Sector'] == 'Operaciones' or row['Sector'] == 'Cultura' or row['Sector'] == 'Público' or row['Sector'] == 'Consultoría  de negocios y Tecnología':
        val = 2
    else:
        val = 3
    return val

In [77]:
df['y3'] = df.apply(f2, axis=1)

In [81]:
y3=df.y3

In [78]:
IndustryY3 = df.groupby(['y3']).agg({'ID':'count'}).reset_index()
IndustryY3

,y3,ID
0,0,11
1,1,14
2,2,6
3,3,7


- nan = 0
- Financiero = 1
- Otros = 2
- Servicios = 3

In [82]:
X_dtm = vect.fit_transform(X)
nb = MultinomialNB()
pd.Series(cross_val_score(nb, X_dtm, y3, cv=4)).mean()

0.3722222222222222

Reduciendo en número de clases se logra una mejora en el resutado del score.